In [9]:
import gym
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [16]:
seed = 42
gamma = 0.99
max_step_per_episode = 10000
env = gym.make('CartPole-v1')
env.seed(seed)
eps = np.finfo(np.float32).eps.item()

num_inputs = 4
num_actions = 2
num_hidden = 128

inputs = layers.Input(shape = (num_inputs,))
co1 = layers.Dense(num_hidden, activation='relu')(inputs)
act1 = layers.Dense(num_actions, activation='softmax')(co1)
critic = layers.Dense(1)(co1)

model = keras.Model(inputs=inputs,  outputs = [act1, critic])
optimizer = keras.optimizers.Adam(learning_rate=0.01)
h_loss = keras.losses.Huber()
action_probs_history = []
critic_value_history = []
rewards_history = []
running_reward = 0
episode_count = 0


while True:
    state = env.reset()
    episode_reward = 0
    with tf.GradientTape() as tape:
        for timestep in range(1, max_step_per_episode):
            env.render()
            state = tf.convert_to_tensor(state)
            state = tf.expand_dims(state, 0)
            action_probs, critic_value = model(state)
            critic_value_history.append(critic_value[0,0])
            
            action = np.random.choice(num_actions, p=np.squeeze(action_probs))
            action_probs_history.append(tf.math.log(action_probs[0, action]))
            
            state, reward, done, _ = env.step(action)
            rewards_history.append(reward)
            episode_reward += reward
            
            if done:
                break
                
            running_reward = 0.05 * episode_reward + (1-0.05) * running_reward
            
            returns = []
            discounted_sum = 0
            for r in rewards_history[::-1]:
                discounted_sum = r+gamma * discounted_sum
                returns.insert(0, discounted_sum)
                
                
            returns = np.array(returns)
            returns = (returns - np.mean(returns)) / (np.std(returns)+eps)
            returns = returns.tolist()
            
            history = zip(action_probs_history, critic_value_history, returns)
            actor_losses = []
            critic_losses = []
            for log_prob, value, ret in history:
                
                diff = ret - value
                actor_losses.append(-log_prob * diff)
                
                critic_losses.append(h_loss(tf.expand_dims(value, 0), tf.expand_dims(ret, 0)))
                
            loss_value = sum(actor_losses) + sum(critic_losses)
            grads = tape.gradient(loss_value, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))
            
            
            action_probs_history.clear()
            critic_value_history.clear()
            rewards_history.clear()
    episode_count += 1
    if episode_count % 10 == 0:
        template = "running reward: {:.2f} at episode {}"
        print(template.format(running_reward, episode_count))
        
    if running_reward > 195:
        print('Solved!')
        break

RuntimeError: A non-persistent GradientTape can only be used to compute one set of gradients (or jacobians)

In [17]:
env.close()

In [19]:
import gym
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Configuration parameters for the whole setup
seed = 42
gamma = 0.99  # Discount factor for past rewards
max_steps_per_episode = 10000
env = gym.make("CartPole-v1")  # Create the environment
env.seed(seed)
eps = np.finfo(np.float32).eps.item()  # Smallest number such that 1.0 + eps != 1.0

num_inputs = 4
num_actions = 2
num_hidden = 128

inputs = layers.Input(shape=(num_inputs,))
common = layers.Dense(num_hidden, activation="relu")(inputs)
action = layers.Dense(num_actions, activation="softmax")(common)
critic = layers.Dense(1)(common)

model = keras.Model(inputs=inputs, outputs=[action, critic])
optimizer = keras.optimizers.Adam(learning_rate=0.01)
h_loss = keras.losses.Huber()
action_probs_history = []
critic_value_history = []
rewards_history = []
running_reward = 0
episode_count = 0

while True:  # Run until solved
    state = env.reset()
    episode_reward = 0
    with tf.GradientTape() as tape:
        for timestep in range(1, max_steps_per_episode):
            env.render()#; Adding this line would show the attempts
            # of the agent in a pop up window.

            state = tf.convert_to_tensor(state)
            state = tf.expand_dims(state, 0)

            # Predict action probabilities and estimated future rewards
            # from environment state
            action_probs, critic_value = model(state)
            critic_value_history.append(critic_value[0, 0])

            # Sample action from action probability distribution
            action = np.random.choice(num_actions, p=np.squeeze(action_probs))
            action_probs_history.append(tf.math.log(action_probs[0, action]))

            # Apply the sampled action in our environment
            state, reward, done, _ = env.step(action)
            rewards_history.append(reward)
            episode_reward += reward

            if done:
                break

        # Update running reward to check condition for solving
        running_reward = 0.05 * episode_reward + (1 - 0.05) * running_reward

        # Calculate expected value from rewards
        # - At each timestep what was the total reward received after that timestep
        # - Rewards in the past are discounted by multiplying them with gamma
        # - These are the labels for our critic
        returns = []
        discounted_sum = 0
        for r in rewards_history[::-1]:
            discounted_sum = r + gamma * discounted_sum
            returns.insert(0, discounted_sum)

        # Normalize
        returns = np.array(returns)
        returns = (returns - np.mean(returns)) / (np.std(returns) + eps)
        returns = returns.tolist()

        history = zip(action_probs_history, critic_value_history, returns)
        actor_losses = []
        critic_losses = []
        for log_prob, value, ret in history:

            diff = ret - value
            actor_losses.append(-log_prob * diff)

            critic_losses.append(h_loss(tf.expand_dims(value, 0), tf.expand_dims(ret, 0)))

        loss_value = sum(actor_losses) + sum(critic_losses)
        grads = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        
        

        action_probs_history.clear()
        critic_value_history.clear()
        rewards_history.clear()
    episode_count += 1
    if episode_count % 10 == 0:
        template = "running reward: {:.2f} at episode {}"
        print(template.format(running_reward, episode_count))

    if running_reward > 195:
        print("Solved at episode {}!".format(episode_count))
        break

running reward: 7.28 at episode 10
running reward: 8.70 at episode 20
running reward: 11.98 at episode 30
running reward: 15.68 at episode 40
running reward: 18.88 at episode 50
running reward: 28.15 at episode 60
running reward: 23.95 at episode 70
running reward: 21.03 at episode 80
running reward: 33.99 at episode 90
running reward: 48.45 at episode 100


KeyboardInterrupt: 

In [20]:
env.close()